In [41]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [42]:
##---------------DATOS DE ENTRENAMIENTO -------------------------##
##Cargar los datos con extención .mat
x = loadmat('/DatosPrueba4.mat')
##Traernos los datos del archivo .mat que nos interesan
y = x['data_tr']
##Convertir los datos en un array de Numpy
z = np.array(y)
##Convertir el Array en un DataFrame de Pandas
df = pd.DataFrame(z)
##Contar el numero de datos nulos del dataFrame
np.count_nonzero(pd.isnull(df) == True)
df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','Label']
df.sample(8)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Label
237,0.002655,0.008684,0.000350,0.000042,4.813048,24.497288,0.060544,0.000137,0.009039,1438.62,441.847082,1676.0,0.0,-2.073042,2.888426,1676.0,222.0,1504.295204,0.0
769,0.002303,0.011805,0.000342,0.000049,7.539154,58.880346,0.105215,0.000117,0.011970,1418.48,253.534389,1476.0,0.0,-4.147060,15.570392,1476.0,222.0,1440.736756,0.0
1060,0.001110,0.002231,0.000321,0.000050,7.056246,59.284337,0.020922,0.000258,0.002482,1450.92,176.444439,1476.0,0.0,-6.754543,44.064802,1476.0,222.0,1461.502706,0.0
787,0.276889,0.653789,0.071026,0.105030,3.623528,13.567729,3.902354,0.000127,0.706989,316.74,122.501092,276.0,0.0,6.185859,47.612656,1348.0,228.0,339.382793,0.0
38,0.000192,0.000388,0.000002,0.000001,2.047976,2.464774,0.001242,0.000000,0.000431,123.91,35.731347,138.0,0.0,-1.045733,-0.668650,158.0,62.0,128.909464,1.0
1039,0.001169,0.000512,0.001005,0.000137,0.527242,1.498700,0.003162,0.000242,0.001276,1335.20,340.104797,1448.0,0.0,-2.626767,4.949700,1448.0,320.0,1377.415551,0.0
579,0.002872,0.006428,0.000336,0.000080,3.725114,15.322742,0.041465,0.000267,0.007011,1402.92,291.390872,1476.0,0.0,-3.676444,11.694172,1476.0,222.0,1432.565559,0.0
128,0.000147,0.000146,0.000124,0.000084,1.842631,3.656517,0.000697,0.000003,0.000207,87.37,37.335946,62.0,0.0,0.790284,-1.341285,153.0,62.0,94.939718,1.0


In [43]:
YEntrenamiento = df['Label']
XEntrenamiento = scaler.fit_transform(df.drop(['Label'],axis=1))

In [44]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE

In [45]:
#ejercicio de código
def entrenamiento_sin_seleccion_caracteristicas(splits, X, Y):
    """
    Función que ejecuta el entrenamiento del modelo sin una selección particular
    de las características

      Parámetros:splits : numero de particiones  a realizar
      Retorna:
      1. El modelo entreando
      2. El vector de errores
      3. El Intervalo de confianza
      4. El tiempo de procesamiento
    """
    #Implemetamos la metodología de validación
    Errores = np.ones(splits)
    Score = np.ones(splits)
    times = np.ones(splits)
    j = 0
    kf = KFold(n_splits=splits)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        #Creamos el clasificador SVM.
        clf = SVC(kernel="linear", C=1)
        #Aquí se entran y se valida el modelo sin hacer selección de características
        ######
        # Entrenamiento el modelo.
        #Para calcular el costo computacional
        tiempo_i = time.time()
        clf.fit(X_train,y_train)
        # Validación del modelo
        Errores[j] = accuracy_score(y_true=y_test, y_pred=clf.predict(X_test))
        times[j] = time.time()-tiempo_i
        j+=1

    return clf, np.mean(Errores), np.std(Errores), np.mean(times)

In [46]:
def recursive_feature_elimination_wrapper(estimator, feature_numbers, X,Y):
    """
    Esta función es un envoltorio del objeto RFE de sklearn

    Parámetros:
    estimator(sklearn.svm.SVC), El estimador SVM
    feature_numbers(int), El número de características a considerar
    X (numpy.array), El arreglo numpy de características
    Y (numpy.array), El vector de etiquetas

    Retorna:
    El modelo entrenado ()
    La máscara de características seleccionada, array [longitud de caracterisitcas de X]
    El rankeo de características, array [longitud de caracterisitcas de X]
    El objeto RFE entrenado sobre el set reducido de características
    El tiempo de ejecución
    """
    rfe = RFE(estimator=estimator, n_features_to_select= feature_numbers, step= 1)
    tiempo_i = time.time()
    rfe.fit(X = X, y =Y)
    time_o = time.time()-tiempo_i
    feature_mask = rfe.support_
    features_rank = rfe.ranking_
    estimator = rfe.estimator_

    return rfe, feature_mask, features_rank, estimator, time_o

In [47]:
def experimentar(n_feats, n_sets, X, Y):
    """
    Esta función realiza la comparación del desempeño de RFE utilizando diferente 
    número de feats y particionando el conjunto de datos en diferente número de 
    subconjuntos

    Parámetros:
    X (numpy.array), El arreglo numpy de características
    Y (numpy.array), El vector de etiquetas
    n_feats, Vector de números enteros que indica el número de características
              que debe utilizar el modelo
    n_sets, Vector de números enteros que indica el número de particiones

    Retorna:  
    - DataFrame con las columnas: CON_SEL, NUM_VAR, NUM_SPLITS, ERROR_VALIDACION, IC_STD_VALIDACION, 
    y T_EJECUCION. 

    """
    df = pd.DataFrame()
    idx = 0
    for split_number in n_sets: 
    #Sin selección de características
        # se ignorar las otras salidas
        _,err,ic,t_ex = entrenamiento_sin_seleccion_caracteristicas(split_number, X,Y)  
        df.loc[idx,'CON_SEL'] = 'NO'
        df.loc[idx,'NUM_VAR'] = X.shape[1]
        df.loc[idx,'NUM_SPLITS'] = split_number
        df.loc[idx,'ERROR_VALIDACION'] = err
        df.loc[idx,'IC_STD_VALIDACION'] = ic
        df.loc[idx,'T_EJECUCION'] = t_ex
        idx+=1
    print("termina experimentos sin selección")
    #Con selección de características
    for f in n_feats:
        for split_number in n_sets:
            #Implemetamos la metodología de validación 
            Errores = np.ones(split_number)
            Score = np.ones(split_number)
            times = np.ones(split_number)
            kf = KFold(n_splits=split_number)
            j = 0
            for train_index, test_index in kf.split(X):
                
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = Y[train_index], Y[test_index]
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
                
                svc =  SVC(kernel="linear", C=1)
                
                # se ignorar las otras salidas
                rfe, _, _, _, t = recursive_feature_elimination_wrapper(estimator=svc,feature_numbers=f, X=X_train,Y=y_train)
            
                Errores[j]=accuracy_score(y_true = y_test,  y_pred=rfe.predict(X_test))
                times[j] = t
                j+=1

            df.loc[idx,'CON_SEL'] = 'SI'
            df.loc[idx,'NUM_VAR'] = f
            df.loc[idx,'NUM_SPLITS'] = split_number
            df.loc[idx,'ERROR_VALIDACION'] = np.mean(Errores)
            df.loc[idx, 'IC_STD_VALIDACION'] = np.std(Errores)
            df.loc[idx, 'T_EJECUCION'] = np.mean(times)
            idx+=1
    return df

In [48]:
dfr = experimentar(n_feats = [3, 5, 10,15,20], n_sets = [5, 10], X= XEntrenamiento, Y=YEntrenamiento)

termina experimentos sin selección


In [49]:
dfr.sort_values(['ERROR_VALIDACION','T_EJECUCION'], ascending=[False, True])

,CON_SEL,NUM_VAR,NUM_SPLITS,ERROR_VALIDACION,IC_STD_VALIDACION,T_EJECUCION
1,NO,18.0,10.0,1.0,0.0,0.002284
11,SI,20.0,10.0,1.0,0.0,0.002309
10,SI,20.0,5.0,1.0,0.0,0.002323
0,NO,18.0,5.0,1.0,0.0,0.002824
8,SI,15.0,5.0,1.0,0.0,0.008961
9,SI,15.0,10.0,1.0,0.0,0.009228
6,SI,10.0,5.0,1.0,0.0,0.018751
7,SI,10.0,10.0,1.0,0.0,0.018802
4,SI,5.0,5.0,1.0,0.0,0.025168
5,SI,5.0,10.0,1.0,0.0,0.025296
